In [1]:
# First make an encryptor object which seeds itself with an unknown seed

from random import randint, choice, choices

from MersenneTwister import MersenneTwister

class Encryptor(object):
    def __init__(self, seed):
        self.seed = seed
    def __call__(self, plaintext: bytes):
        twister = MersenneTwister(self.seed)
        assert type(plaintext) is bytes
        ciphertext = [None] * len(plaintext)
        for n in range(len(plaintext)):
            key = twister() & 0xFF
            ciphertext[n] = key ^ plaintext[n]
        return bytes(ciphertext)
        

In [2]:
encryptor = Encryptor(123)

In [3]:
plaintext = b'But soft, what light through yonder window breaks?'
ciphertext = encryptor(plaintext)

In [4]:
ciphertext

b"D\x983\x02\xdcr\xda\x7f|M\xaa\x91w\xb2+\xe9,&'\xe9h\xa6^fckn\x0b8\xd2\xadm\x8dj\x90\n\\UNg\xffZ\xa0\xf4\xe9\x03\xff\xfb\x84u"

In [5]:
# Proper decrypt
assert plaintext == encryptor(ciphertext)

In [6]:
from time import time

# Create an encryptor with a 16 bit seed derived from the time
# and encrypt a know value
encryptor = Encryptor( int(time()) % (0b1<<16) )
plaintext = bytes( choices(range(0xFF), k = randint(1, 5)) ) + b'A'*14

print(plaintext)

ciphertext = encryptor(plaintext)

print(ciphertext)

assert encryptor(ciphertext) == plaintext

b'\xedAAAAAAAAAAAAAA'
b'\x97eF\x87\x95T\x1b\xf4\x8f8\xa1\xf9\xd6\xc9\xed'


In [7]:
# Now to break it...
# I thought we wouuld use some variation of the attack in the last exercise.
# But it turns out that because it's only a 16 bit seed you can brute force it.
# This seems cheap... but the instructions say 16 bit specifcally...
# I looked at others who have posted solutions to these exercises and all agree that brute force is the proper solution.
# I am a bit anxious to wrap up this set so I'm just going to do that as well as the easy solution.

for seed in range(0b1<<17):
    if Encryptor(seed)(ciphertext) == plaintext:
        print(f'Seed is {seed}')
        break
        
assert encryptor.seed == seed

Seed is 11100


In [8]:
# Now do the password reset token thing

def GenerateToken(username: bytes):
    prefix = b'\nW\xb2\x1eIHK\xaf \x99\xd7\x0e\xf1Mm\x83'
    plaintext = prefix + username
    encryptor = Encryptor( int(time()) % (0b1<<16) )
    return encryptor(plaintext)

In [9]:
username = b'john.q.public'

ciphertext = GenerateToken(username)
ciphertext

b'\xaeJ\x88M\xeb$\xf9\xff\xc2w\xeb;;\xe2\x81\xb8A\x9d^#\x0b\xf4\xd4\x91\xb3\xbf\xbd\xc1\xb0'

In [10]:
# Now brute force it

for seed in range(0b1<<16):
    plaintext = Encryptor(seed)(ciphertext)
    if username in plaintext:
        print(f'Seed is {seed}')
        print(f'Plaintext is {plaintext}')
        break

Seed is 11121
Plaintext is b'\nW\xb2\x1eIHK\xaf \x99\xd7\x0e\xf1Mm\x83john.q.public'
